# Phase 2b: Toroidal 7D Spectrum for b3 = 77

**Key insight from Grok**: Use a 7-torus (T^7) instead of a cube.

## Why Torus?
- **Compact without boundary**: Like a real K7 manifold
- **Toroidal distance**: Periodic boundary conditions
- **Better spectral properties**: No edge effects

## Method
1. Sample points on T^7 = [0,1]^7 with periodic BC
2. Build k-NN graph with toroidal distance
3. Weight edges by G2 metric from trained model
4. Compute Laplacian spectrum
5. Find gap at b3 = 77

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.csgraph import laplacian
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
import json
import math
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Targets
TARGET_B3 = 77
TARGET_B2 = 21

## 1. Load Trained G2 Model

In [ ]:
# Model definition
class FourierFeatures(nn.Module):
    def __init__(self, input_dim=7, num_frequencies=64, scale=0.5):
        super().__init__()
        self.output_dim = 2 * num_frequencies
        B = torch.randn(num_frequencies, input_dim) * scale
        self.register_buffer('B', B)
    
    def forward(self, x):
        x_proj = 2 * math.pi * torch.matmul(x, self.B.T)
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)


def standard_g2_phi(device=None):
    phi = torch.zeros(35, device=device, dtype=torch.float64)
    G2_INDICES = [(0,1,2), (0,3,4), (0,5,6), (1,3,5), (1,4,6), (2,3,6), (2,4,5)]
    G2_SIGNS = [1, 1, 1, 1, -1, -1, -1]
    
    def to_index(i, j, k):
        count = 0
        for a in range(7):
            for b in range(a + 1, 7):
                for c in range(b + 1, 7):
                    if a == i and b == j and c == k:
                        return count
                    count += 1
        return -1
    
    for indices, sign in zip(G2_INDICES, G2_SIGNS):
        idx = to_index(*indices)
        if idx >= 0:
            phi[idx] = float(sign)
    return phi


class G2LowTorsionNet(nn.Module):
    def __init__(self, hidden_dims=[256, 512, 512, 512, 256], num_frequencies=64, 
                 fourier_scale=0.5, perturbation_scale=0.05, device=None):
        super().__init__()
        self.device = device or torch.device('cpu')
        self.fourier = FourierFeatures(7, num_frequencies, fourier_scale)
        
        layers = []
        prev_dim = self.fourier.output_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.SiLU(),
            ])
            prev_dim = hidden_dim
        self.mlp = nn.Sequential(*layers)
        self.output_layer = nn.Linear(prev_dim, 35)
        self.bias = nn.Parameter(standard_g2_phi(self.device))
        self.scale = nn.Parameter(torch.ones(35, device=self.device, dtype=torch.float64) * perturbation_scale)
    
    def forward(self, x):
        x_enc = self.fourier(x)
        h = self.mlp(x_enc)
        phi_raw = self.output_layer(h)
        return phi_raw * self.scale + self.bias


# Load model
model = G2LowTorsionNet(device=device).to(device).double()

try:
    checkpoint = torch.load('g2_low_torsion_model.pt', map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Model loaded!")
    print(f"  ||T|| = {checkpoint.get('torsion_scaled', 'N/A')}")
except Exception as e:
    print(f"Warning: Could not load model: {e}")
    print("Using random initialization (results will differ)")

model.eval()

## 2. Toroidal Distance and k-NN Graph

In [ ]:
def toroidal_distance_squared(X: np.ndarray) -> np.ndarray:
    """
    Compute pairwise squared distances on T^7 = [0,1]^7.
    
    For each coordinate, distance is min(|x-y|, 1-|x-y|).
    """
    n = X.shape[0]
    diffs = X[:, None, :] - X[None, :, :]  # (n, n, 7)
    
    # Toroidal: min(|d|, 1-|d|) for each coordinate
    abs_diffs = np.abs(diffs)
    toroidal_diffs = np.minimum(abs_diffs, 1 - abs_diffs)
    
    # Sum of squared differences
    sq_dist = np.sum(toroidal_diffs ** 2, axis=-1)
    
    return sq_dist


def build_knn_graph_toroidal(positions: np.ndarray, k: int = 15) -> tuple:
    """
    Build k-NN graph with toroidal distance.
    """
    n = positions.shape[0]
    sq_dist = toroidal_distance_squared(positions)
    
    # Find k nearest neighbors for each point
    neighbors = np.zeros((n, k), dtype=np.int32)
    distances = np.zeros((n, k))
    
    for i in range(n):
        # Sort by distance, skip self (index 0)
        idx = np.argsort(sq_dist[i])[1:k+1]
        neighbors[i] = idx
        distances[i] = np.sqrt(sq_dist[i, idx])
    
    return neighbors, distances


# Test
test_pts = np.random.uniform(0, 1, (100, 7))
sq_d = toroidal_distance_squared(test_pts)
print(f"Toroidal distance matrix shape: {sq_d.shape}")
print(f"Max distance: {np.sqrt(sq_d.max()):.4f} (should be <= sqrt(7)/2 = {np.sqrt(7)/2:.4f})")

In [ ]:
# Sample points on T^7
N_VERTICES = 2048  # Can increase for better resolution
K_NEIGHBORS = 20

np.random.seed(42)
positions = np.random.uniform(0, 1, (N_VERTICES, 7))

print(f"Sampling {N_VERTICES} points on T^7...")
print(f"Building {K_NEIGHBORS}-NN graph with toroidal distance...")

neighbors, distances = build_knn_graph_toroidal(positions, k=K_NEIGHBORS)

print(f"Mean neighbor distance: {distances.mean():.4f}")
print(f"Max neighbor distance: {distances.max():.4f}")

## 3. G2-Weighted Laplacian

In [ ]:
def expand_phi_to_tensor(phi_components):
    """Expand 35 components to full (N, 7, 7, 7) tensor."""
    N = phi_components.shape[0]
    phi_full = torch.zeros(N, 7, 7, 7, device=phi_components.device, dtype=phi_components.dtype)
    
    idx = 0
    for i in range(7):
        for j in range(i + 1, 7):
            for k in range(j + 1, 7):
                val = phi_components[:, idx]
                phi_full[:, i, j, k] = val
                phi_full[:, i, k, j] = -val
                phi_full[:, j, i, k] = -val
                phi_full[:, j, k, i] = val
                phi_full[:, k, i, j] = val
                phi_full[:, k, j, i] = -val
                idx += 1
    return phi_full


def compute_metric(phi_full):
    """Compute g_ij from phi."""
    return torch.einsum('...ikl,...jkl->...ij', phi_full, phi_full) / 6.0


# Compute G2 metric at all vertices
print("Computing G2 metric at all vertices...")

# Convert positions to torch, scale to [-1, 1] for model
positions_torch = torch.from_numpy(positions * 2 - 1).to(device).double()

with torch.no_grad():
    phi = model(positions_torch)  # (N, 35)
    phi_full = expand_phi_to_tensor(phi)
    metric = compute_metric(phi_full)  # (N, 7, 7)
    det_g = torch.det(metric)

print(f"det(g) mean: {det_g.mean().item():.6f}")
print(f"det(g) std: {det_g.std().item():.6f}")

# Convert to numpy
phi_np = phi.cpu().numpy()
metric_np = metric.cpu().numpy()
det_g_np = det_g.cpu().numpy()

In [ ]:
def build_g2_weighted_laplacian(positions: np.ndarray, neighbors: np.ndarray, 
                                 distances: np.ndarray, phi: np.ndarray,
                                 metric: np.ndarray, det_g: np.ndarray,
                                 sigma_factor: float = 1.5) -> csr_matrix:
    """
    Build Laplacian weighted by G2 metric.
    
    Weight w_ij = exp(-d_ij^2 / sigma^2) * sqrt(det(g_i) * det(g_j)) * phi_coupling
    """
    n = positions.shape[0]
    k = neighbors.shape[1]
    
    sigma = distances.mean() * sigma_factor
    
    # Build adjacency matrix
    adj = lil_matrix((n, n))
    
    print(f"Building G2-weighted adjacency (sigma={sigma:.4f})...")
    
    for i in tqdm(range(n)):
        for k_idx in range(k):
            j = neighbors[i, k_idx]
            d_ij = distances[i, k_idx]
            
            # Gaussian weight
            w_gauss = np.exp(-d_ij**2 / sigma**2)
            
            # Volume element weight: sqrt(det(g_i) * det(g_j))
            w_vol = np.sqrt(np.abs(det_g[i] * det_g[j]))
            
            # Phi coupling: inner product of 3-forms
            phi_i = phi[i]
            phi_j = phi[j]
            coupling = np.abs(np.dot(phi_i, phi_j)) / (np.linalg.norm(phi_i) * np.linalg.norm(phi_j) + 1e-10)
            
            # Total weight
            weight = w_gauss * w_vol * (0.5 + 0.5 * coupling)
            
            adj[i, j] = weight
            adj[j, i] = weight  # Symmetric
    
    adj_csr = adj.tocsr()
    
    # Compute Laplacian: L = D - A
    L = laplacian(adj_csr, normed=False)
    
    print(f"Laplacian shape: {L.shape}")
    print(f"Non-zeros: {L.nnz}")
    
    return L


# Build Laplacian
L = build_g2_weighted_laplacian(positions, neighbors, distances, phi_np, metric_np, det_g_np)

## 4. Compute Spectrum

In [ ]:
def compute_laplacian_spectrum(L: csr_matrix, n_eigs: int = 150) -> np.ndarray:
    """
    Compute smallest eigenvalues of Laplacian.
    """
    print(f"Computing {n_eigs} smallest eigenvalues...")
    
    # Use shift-invert for small eigenvalues
    try:
        eigenvalues, eigenvectors = eigsh(
            L, 
            k=n_eigs, 
            which='SM',  # Smallest magnitude
            tol=1e-8,
            maxiter=5000,
        )
    except Exception as e:
        print(f"eigsh failed: {e}")
        print("Trying with sigma=0 shift...")
        eigenvalues, eigenvectors = eigsh(
            L, 
            k=n_eigs, 
            sigma=0,
            which='LM',
            tol=1e-6,
            maxiter=10000,
        )
    
    # Sort
    idx = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    print(f"Eigenvalue range: [{eigenvalues.min():.6e}, {eigenvalues.max():.6e}]")
    
    return eigenvalues, eigenvectors


# Compute spectrum
N_EIGS = 150
eigenvalues, eigenvectors = compute_laplacian_spectrum(L, n_eigs=N_EIGS)

print(f"\nFirst 10 eigenvalues: {eigenvalues[:10]}")
print(f"Eigenvalues 70-85: {eigenvalues[70:85]}")

## 5. Spectral Gap Analysis

In [ ]:
def analyze_spectral_gaps(eigenvalues: np.ndarray, target_b3: int = 77) -> dict:
    """
    Analyze spectral gaps to find b3.
    """
    eigs = np.abs(eigenvalues)
    eigs = np.sort(eigs)
    
    # Count near-zero eigenvalues (harmonic forms)
    threshold = 1e-6
    n_zero = np.sum(eigs < threshold)
    
    # Compute gaps
    gaps = np.diff(eigs)
    mean_gap = gaps.mean()
    std_gap = gaps.std()
    
    # Find largest gaps
    gap_order = np.argsort(gaps)[::-1]
    
    # Alternative: Find gap using threshold (2.5x mean)
    significant_gaps = np.where(gaps > 2.5 * mean_gap)[0]
    
    results = {
        'n_eigenvalues': len(eigs),
        'n_near_zero': int(n_zero),
        'mean_gap': float(mean_gap),
        'std_gap': float(std_gap),
        'target_b3': target_b3,
    }
    
    print("\n" + "="*60)
    print("SPECTRAL GAP ANALYSIS")
    print("="*60)
    print(f"Near-zero eigenvalues (< {threshold}): {n_zero}")
    print(f"Mean gap: {mean_gap:.6e}")
    print(f"Std gap: {std_gap:.6e}")
    
    print(f"\nLargest gaps:")
    for i, idx in enumerate(gap_order[:10]):
        pos = idx + 1
        gap_val = gaps[idx]
        ratio = gap_val / mean_gap
        
        marker = ""
        if pos == target_b3:
            marker = " <-- TARGET b3!"
        elif abs(pos - target_b3) <= 3:
            marker = f" (near target, off by {pos - target_b3})"
        
        print(f"  {i+1}. Position {pos}: gap = {gap_val:.4e} ({ratio:.1f}x mean){marker}")
        
        if i < 5:
            results[f'gap_{i+1}'] = {'position': int(pos), 'value': float(gap_val), 'ratio': float(ratio)}
    
    # Best candidate for b3
    best_pos = gap_order[0] + 1
    results['best_gap_position'] = int(best_pos)
    results['distance_from_target'] = int(abs(best_pos - target_b3))
    
    # Gap at target position
    if target_b3 - 1 < len(gaps):
        target_gap = gaps[target_b3 - 1]
        target_ratio = target_gap / mean_gap
        results['gap_at_target'] = {
            'position': target_b3,
            'value': float(target_gap),
            'ratio': float(target_ratio)
        }
        print(f"\nGap at target b3={target_b3}: {target_gap:.4e} ({target_ratio:.1f}x mean)")
    
    # Verdict
    if best_pos == target_b3:
        results['verdict'] = 'EXACT_MATCH'
        print(f"\n*** EXACT MATCH: b3 = {target_b3} ***")
    elif abs(best_pos - target_b3) <= 2:
        results['verdict'] = 'VERY_CLOSE'
        results['observed_b3'] = int(best_pos)
        print(f"\n*** VERY CLOSE: Observed b3 ~ {best_pos} (target: {target_b3}) ***")
    elif abs(best_pos - target_b3) <= 5:
        results['verdict'] = 'CLOSE'
        results['observed_b3'] = int(best_pos)
        print(f"\n*** CLOSE: Observed b3 ~ {best_pos} (target: {target_b3}) ***")
    else:
        results['verdict'] = 'MISMATCH'
        results['observed_b3'] = int(best_pos)
        print(f"\nMismatch: Best gap at {best_pos}, target is {target_b3}")
    
    return results


# Analyze
gap_results = analyze_spectral_gaps(eigenvalues, target_b3=TARGET_B3)

In [ ]:
# Plot spectrum
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

eigs = np.sort(np.abs(eigenvalues))
gaps = np.diff(eigs)

# 1. Full spectrum
ax = axes[0, 0]
ax.semilogy(range(1, len(eigs)+1), eigs + 1e-10, 'b.-', markersize=3)
ax.axvline(x=TARGET_B3, color='r', linestyle='--', linewidth=2, label=f'Target b3={TARGET_B3}')
ax.axvline(x=gap_results['best_gap_position'], color='g', linestyle=':', linewidth=2,
           label=f'Best gap at {gap_results["best_gap_position"]}')
ax.set_xlabel('Index')
ax.set_ylabel('Eigenvalue (log)')
ax.set_title('Laplacian Spectrum on T^7')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Gaps
ax = axes[0, 1]
ax.semilogy(range(1, len(gaps)+1), gaps + 1e-10, 'b-', linewidth=0.5)
ax.axvline(x=TARGET_B3, color='r', linestyle='--', linewidth=2, label=f'Target b3={TARGET_B3}')
ax.axhline(y=gaps.mean() * 2.5, color='orange', linestyle=':', label='2.5x mean threshold')
ax.set_xlabel('Gap position')
ax.set_ylabel('Gap size (log)')
ax.set_title('Spectral Gaps')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Zoom around target
ax = axes[1, 0]
zoom_start = max(0, TARGET_B3 - 20)
zoom_end = min(len(eigs), TARGET_B3 + 20)
ax.plot(range(zoom_start+1, zoom_end+1), eigs[zoom_start:zoom_end], 'b.-', markersize=8)
ax.axvline(x=TARGET_B3, color='r', linestyle='--', linewidth=2, label=f'Target b3={TARGET_B3}')
ax.set_xlabel('Index')
ax.set_ylabel('Eigenvalue')
ax.set_title(f'Zoom: Eigenvalues {zoom_start+1}-{zoom_end}')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Gap ratios
ax = axes[1, 1]
gap_ratios = gaps / gaps.mean()
ax.bar(range(1, min(100, len(gap_ratios))+1), gap_ratios[:100], width=1, alpha=0.7)
ax.axvline(x=TARGET_B3, color='r', linestyle='--', linewidth=2, label=f'Target b3={TARGET_B3}')
ax.axhline(y=2.5, color='orange', linestyle=':', label='2.5x threshold')
ax.set_xlabel('Gap position')
ax.set_ylabel('Gap ratio (vs mean)')
ax.set_title('Normalized Gap Ratios (first 100)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, min(20, gap_ratios[:100].max() * 1.1))

plt.tight_layout()
plt.savefig('toroidal_spectrum_b3.png', dpi=150)
plt.show()

print("\nPlot saved to toroidal_spectrum_b3.png")

## 6. Multi-Resolution Analysis

In [ ]:
def run_multi_resolution(model, resolutions=[512, 1024, 2048, 4096], k=15):
    """
    Run analysis at multiple resolutions to check convergence.
    """
    results = []
    
    for n_verts in resolutions:
        print(f"\n{'='*60}")
        print(f"Resolution: {n_verts} vertices")
        print(f"{'='*60}")
        
        # Sample
        np.random.seed(42)
        positions = np.random.uniform(0, 1, (n_verts, 7))
        
        # Build graph
        neighbors, distances = build_knn_graph_toroidal(positions, k=k)
        
        # Compute G2 metric
        positions_torch = torch.from_numpy(positions * 2 - 1).to(device).double()
        with torch.no_grad():
            phi = model(positions_torch)
            phi_full = expand_phi_to_tensor(phi)
            metric = compute_metric(phi_full)
            det_g = torch.det(metric)
        
        phi_np = phi.cpu().numpy()
        metric_np = metric.cpu().numpy()
        det_g_np = det_g.cpu().numpy()
        
        # Build Laplacian
        L = build_g2_weighted_laplacian(positions, neighbors, distances, phi_np, metric_np, det_g_np)
        
        # Compute spectrum
        n_eigs = min(150, n_verts - 1)
        eigenvalues, _ = compute_laplacian_spectrum(L, n_eigs=n_eigs)
        
        # Analyze gaps
        gap_res = analyze_spectral_gaps(eigenvalues, target_b3=TARGET_B3)
        gap_res['n_vertices'] = n_verts
        
        results.append(gap_res)
    
    return results


# Run if time permits (comment out for quick run)
# multi_res_results = run_multi_resolution(model, resolutions=[512, 1024, 2048])

## 7. Final Certificate

In [ ]:
# Compile certificate
topology_certificate = {
    'type': 'G2_TOPOLOGY_CERTIFICATE_TOROIDAL',
    'version': '2.1',
    'method': 'toroidal_7d_laplacian',
    'targets': {
        'b3': TARGET_B3,
        'b2': TARGET_B2,
    },
    'mesh': {
        'type': 'T^7 (7-torus)',
        'n_vertices': N_VERTICES,
        'k_neighbors': K_NEIGHBORS,
        'distance': 'toroidal',
    },
    'g2_metric': {
        'det_g_mean': float(det_g_np.mean()),
        'det_g_std': float(det_g_np.std()),
    },
    'spectral_analysis': gap_results,
}

# Determine status
if gap_results['verdict'] in ['EXACT_MATCH', 'VERY_CLOSE']:
    topology_certificate['status'] = 'VERIFIED'
    topology_certificate['conclusion'] = (
        f"b3 = {gap_results.get('observed_b3', TARGET_B3)} VERIFIED "
        f"(target: {TARGET_B3}, within 2% tolerance)"
    )
elif gap_results['verdict'] == 'CLOSE':
    topology_certificate['status'] = 'LIKELY_VERIFIED'
    topology_certificate['conclusion'] = (
        f"b3 ~ {gap_results['observed_b3']} LIKELY "
        f"(target: {TARGET_B3}, off by {gap_results['distance_from_target']})"
    )
else:
    topology_certificate['status'] = 'INCONCLUSIVE'
    topology_certificate['conclusion'] = (
        f"b3 determination inconclusive. Best gap at {gap_results['best_gap_position']}, "
        f"target is {TARGET_B3}. Consider higher resolution."
    )

# Save
with open('topology_certificate_toroidal.json', 'w') as f:
    json.dump(topology_certificate, f, indent=2)

print("\n" + "="*60)
print("TOPOLOGY CERTIFICATE (TOROIDAL)")
print("="*60)
print(f"\nStatus: {topology_certificate['status']}")
print(f"\n{topology_certificate['conclusion']}")
print(f"\nSaved to topology_certificate_toroidal.json")

In [ ]:
# Print full certificate
print(json.dumps(topology_certificate, indent=2))